In [7]:
from pymongo import MongoClient
# Connect to a local MongoDB instance
client = MongoClient('mongodb://localhost:27017/')
db = client['mflix']
comment_collection = db.get_collection('comments')

i. Find top 10 users who made the maximum number of comments

In [4]:
# Aggregate to find top 10 users with the maximum number of comments
pipeline = [
    {"$group": {"_id": "$email", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

top_users = list(comment_collection.aggregate(pipeline))

# Print the top 10 users
for idx, user in enumerate(top_users, start=1):
    print(f"#{idx}: User '{user['_id']}' made {user['count']} comments.")

#1: User 'roger_ashton-griffiths@gameofthron.es' made 331 comments.
#2: User 'nathalie_emmanuel@gameofthron.es' made 327 comments.
#3: User 'jonathan_pryce@gameofthron.es' made 315 comments.
#4: User 'sophie_turner@gameofthron.es' made 308 comments.
#5: User 'ron_donachie@gameofthron.es' made 305 comments.
#6: User 'robert_jordan@fakegmail.com' made 304 comments.
#7: User 'paul_kaye@gameofthron.es' made 304 comments.
#8: User 'gwendoline_christie@gameofthron.es' made 302 comments.
#9: User 'kathryn_sosa@fakegmail.com' made 296 comments.
#10: User 'andrea_le@fakegmail.com' made 296 comments.


ii. Find top 10 movies with most comments


In [13]:
movie_collection = db.get_collection('movies')

# Aggregate to find top 10 movies with the most comments
pipeline = [
    {"$group": {"_id": "$movie_id", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

top_movies_with_comments = list(comment_collection.aggregate(pipeline))

# Get movie details for top movies
top_movies = []
for movie in top_movies_with_comments:
    movie_details = movie_collection.find_one({"_id": movie['_id']})
    if movie_details:
        top_movies.append({"title": movie_details['title'], "comment_count": movie['count']})

# Print the top 10 movies with most comments
for idx, movie in enumerate(top_movies, start=1):
    print(f"#{idx}: Movie '{movie['title']}' has {movie['comment_count']} comments.")

#1: Movie 'The Taking of Pelham 1 2 3' has 161 comments.
#2: Movie 'Ocean's Eleven' has 158 comments.
#3: Movie 'Terminator Salvation' has 158 comments.
#4: Movie 'About a Boy' has 158 comments.
#5: Movie '50 First Dates' has 158 comments.
#6: Movie 'The Mummy' has 157 comments.
#7: Movie 'Sherlock Holmes' has 157 comments.
#8: Movie 'Hellboy II: The Golden Army' has 155 comments.
#9: Movie 'Anchorman: The Legend of Ron Burgundy' has 154 comments.
#10: Movie 'The Mummy Returns' has 154 comments.


iii. Given a year find the total number of comments created each month in that year

In [18]:
from datetime import datetime

# Given year
given_year = 1998  # Update with the desired year

# Aggregate to find total number of comments created each month in the given year
pipeline = [
    {"$match": {"date": {"$gte": datetime(given_year, 1, 1), "$lt": datetime(given_year + 1, 1, 1)}}},
    {"$project": {"month": {"$month": "$date"}}},
    {"$group": {"_id": "$month", "total_comments": {"$sum": 1}}},
    {"$sort": {"_id": 1}}
]

comments_by_month = list(comments_collection.aggregate(pipeline))

# Print the total number of comments created each month in the given year
for month_data in comments_by_month:
    month_number = month_data['_id']
    month_name = datetime(1900, month_number, 1).strftime('%B')  # Get month name from month number
    total_comments = month_data['total_comments']
    print(f"{month_name} {given_year}: {total_comments} comments")


January 1998: 93 comments
February 1998: 77 comments
March 1998: 78 comments
April 1998: 67 comments
May 1998: 104 comments
June 1998: 70 comments
July 1998: 78 comments
August 1998: 105 comments
September 1998: 81 comments
October 1998: 98 comments
November 1998: 75 comments
December 1998: 97 comments
